# Emotion Classification
- In this project i will do the emotion classification pooject.
- The goal of this project is to classifiy emotion from `audio`.

In [2]:
# !pip install --upgrade transformers datasets evaluate accelerate peft torch torchaudio

In [3]:
import torch
from datasets import load_dataset
import transformers

# Login to hugging face

In [4]:
# from huggingface_hub import notebook_login
# notebook_login()

# Load Dataset

In [5]:
emotions = load_dataset("xbgoose/ravdess")

In [6]:
emotions

DatasetDict({
    train: Dataset({
        features: ['audio', 'modality', 'vocal_channel', 'emotion', 'emotional_intensity', 'statement', 'repetition', 'actor', 'gender'],
        num_rows: 1440
    })
})

# Observation
- we can see that data contain multiple columns but we can only 2 columns `audio` and `emotion`

In [7]:
emotions=emotions.remove_columns(['modality','vocal_channel','emotional_intensity','statement','repetition','actor','gender'])

In [8]:
emotions

DatasetDict({
    train: Dataset({
        features: ['audio', 'emotion'],
        num_rows: 1440
    })
})

# Train test split

In [9]:
emotions=emotions['train'].train_test_split(test_size=0.2)

In [10]:
train_df=emotions['train']
test_df=emotions['test']

In [11]:
train_df

Dataset({
    features: ['audio', 'emotion'],
    num_rows: 1152
})

In [12]:
test_df

Dataset({
    features: ['audio', 'emotion'],
    num_rows: 288
})

# Take the look of data

In [13]:
train_df[0]

{'audio': {'path': '03-01-06-02-01-02-03.wav',
  'array': array([-0.01434326, -0.01327515, -0.01361084, ..., -0.00317383,
         -0.00247192, -0.00164795]),
  'sampling_rate': 48000},
 'emotion': 'fearful'}

# Observation
- There are 2 fields `audio` and `emotion`
  - `**Audio**`
    - It's a 1-D array of the speech signal that must be called to load and resample the audio file.
    - Audio contain the information of audio file.
  - `**Emotion**`
    - Emotion contain the emotion of audion file like `angry`,`disgust` etc

# play any of the file

In [14]:
import pandas as pd
import IPython.display as ipd
import random

sample=random.randint(0,len(train_df))
print("Emption",train_df[sample]['emotion'])
ipd.Audio(train_df[sample]['audio']['array'],rate=train_df[sample]['audio']['sampling_rate'])

Emption neutral


# Encode the `target` col

In [15]:
set(train_df['emotion'])

{'angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised'}

In [16]:
# Label to id
label2id={
    "angry":0,
    'calm':1,
    'disgust':2,
    'fearful':3,
    'happy':4,
    'neutral':5,
    'sad':6,
    'surprised':7
}

# id to label
id2label={
    0:"angry",
    1:'calm',
    2:'disgust',
    3:'fearful',
    4:'happy',
    5:'neutral',
    6:'sad',
    7:'surprised'
}

In [17]:
# Now map the label2id in emotion col
train_df=train_df.map(lambda x: {'label':label2id[x['emotion']]})

test_df=test_df.map(lambda x: {'label':label2id[x['emotion']]})

Map:   0%|          | 0/1152 [00:00<?, ? examples/s]

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

In [18]:
train_df['label'][:5]

[3, 2, 1, 7, 5]

In [19]:
train_df['emotion'][:5]

['fearful', 'disgust', 'calm', 'surprised', 'neutral']

# Now Load the `pretrain` processor to process the audion file

In [20]:
# !pip install --upgrade torch torchaudio

In [21]:
import torch
import torchaudio
print(torch.version.cuda)
print(torchaudio.__version__)

12.1
2.4.0+cu121


In [22]:
from transformers import AutoFeatureExtractor
check_point='MIT/ast-finetuned-audioset-10-10-0.4593'
feature_extractor=AutoFeatureExtractor.from_pretrained(check_point)

preprocessor_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [23]:
feature_extractor

ASTFeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "ASTFeatureExtractor",
  "feature_size": 1,
  "max_length": 1024,
  "mean": -4.2677393,
  "num_mel_bins": 128,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000,
  "std": 4.5689974
}

In [23]:
train_df[0]

{'audio': {'path': '03-01-05-02-01-02-01.wav',
  'array': array([-0.00067139,  0.00253296,  0.00317383, ...,  0.        ,
          0.        ,  0.        ]),
  'sampling_rate': 48000},
 'emotion': 'angry',
 'label': 0}

# Observation
- We can see that `feature extractor` expect 16000 sampling rate but in our data sampling rate is 48000 now we can resample the audio.

In [30]:
from datasets import Audio

In [25]:
train_df=train_df.cast_column('audio',Audio(sampling_rate=feature_extractor.sampling_rate))
test_df=test_df.cast_column('audio',Audio(sampling_rate=feature_extractor.sampling_rate))

In [26]:
train_df[0]

{'audio': {'path': '03-01-05-02-01-02-01.wav',
  'array': array([ 1.00730092e-03,  2.65891477e-03, -2.70536821e-03, ...,
         -1.50642109e-07, -2.17821253e-07,  1.79170456e-07]),
  'sampling_rate': 16000},
 'emotion': 'angry',
 'label': 0}

# Observation
- Finally we can resample the audio.
- Now we can make a fun that can take the audio data and make them the model expected format.

# Important
- `Now we can load the model and test the model before fine-tunning.`

In [27]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

model=AutoModelForAudioClassification.from_pretrained(
    check_point,
    num_labels=len(id2label),
    label2id=label2id,
    id2label=id2label,
     ignore_mismatched_sizes=True
)

config.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([8]) in the model instantiated
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([8, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
inputs = feature_extractor(train_df[999]["audio"]["array"], sampling_rate=train_df[999]["audio"]["sampling_rate"], return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])
print("Actual class:", id2label[train_df[0]['label']])

Predicted class: fearful
Actual class: angry


In [29]:
train_df[999]['label']

3

# Observation
- Here we see that model can't predict well in our data.
- Now we can fine tune the model in our dataset

In [30]:
def process_audio(batch):
  # get the audin array
  input_audio=[x['array'] for x in batch['audio']]

  inputs=feature_extractor(input_audio,
                   sampling_rate=feature_extractor.sampling_rate,
                   max_length=16000,truncation=True,return_tensors='pt')

  return inputs

# **Test the fun**

In [31]:
process_audio(train_df[0:5])

{'input_values': tensor([[[-1.2776, -1.2776, -1.2776,  ..., -0.0857, -0.2256, -0.4870],
         [-1.2776, -1.2776, -1.2776,  ..., -0.0686, -0.1137, -0.4669],
         [-1.2776, -1.2776, -1.2776,  ...,  0.0295, -0.2803, -0.7069],
         ...,
         [ 0.4670,  0.4670,  0.4670,  ...,  0.4670,  0.4670,  0.4670],
         [ 0.4670,  0.4670,  0.4670,  ...,  0.4670,  0.4670,  0.4670],
         [ 0.4670,  0.4670,  0.4670,  ...,  0.4670,  0.4670,  0.4670]],

        [[-1.2776, -1.2776, -1.2776,  ..., -1.2776, -1.2776, -1.2776],
         [-1.2776, -1.2776, -1.2776,  ..., -1.2776, -1.2776, -1.2776],
         [-1.2776, -1.2776, -1.2776,  ..., -1.2776, -1.2776, -1.2776],
         ...,
         [ 0.4670,  0.4670,  0.4670,  ...,  0.4670,  0.4670,  0.4670],
         [ 0.4670,  0.4670,  0.4670,  ...,  0.4670,  0.4670,  0.4670],
         [ 0.4670,  0.4670,  0.4670,  ...,  0.4670,  0.4670,  0.4670]],

        [[-1.2776, -1.2776, -1.2776,  ..., -1.2776, -1.2776, -1.2776],
         [-1.2776, -1.2776, 

# Apply the fun to whole data

In [32]:
print("Processing train data...")
train_df=train_df.map(process_audio,batched=True)

print("Processing test data....")
test_df=test_df.map(process_audio,batched=True)

Processing train data...


Map:   0%|          | 0/1152 [00:00<?, ? examples/s]

Processing test data....


Map:   0%|          | 0/288 [00:00<?, ? examples/s]

# Observation
- Finally we can process the whole audio array.
- Now we can get only necessary col and remove other columns

In [33]:
train_df

Dataset({
    features: ['audio', 'emotion', 'label', 'input_values'],
    num_rows: 1152
})

# Remove columns
- Removing columns are `audio`,`emotion`

In [34]:
train_df=train_df.remove_columns(['audio','emotion'])
test_df=test_df.remove_columns(['audio','emotion'])

In [35]:
train_df

Dataset({
    features: ['label', 'input_values'],
    num_rows: 1152
})

# Observation
- Now we can ready to train the model

In [36]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

model=AutoModelForAudioClassification.from_pretrained(
    check_point,
    num_labels=len(id2label),
    label2id=label2id,
    id2label=id2label,
     ignore_mismatched_sizes=True
)

Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([8]) in the model instantiated
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([8, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:

model

ASTForAudioClassification(
  (audio_spectrogram_transformer): ASTModel(
    (embeddings): ASTEmbeddings(
      (patch_embeddings): ASTPatchEmbeddings(
        (projection): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ASTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ASTLayer(
          (attention): ASTSdpaAttention(
            (attention): ASTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ASTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ASTIntermediate(
       

# Load the loss fun for evulation

In [14]:
!pip install --upgrade --force-reinstall torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.9 MB/s eta 0:00:00
  Using cached torch-2.4.0-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-1

In [39]:
import evaluate
import numpy as np

metric=evaluate.load("accuracy")

In [40]:
def compute_matrics(eval_pre):
  prediction=np.argmax(eval_pre.predictions,axis=1)

  return metric.compute(predictions=prediction,references=eval_pre.label_ids)

# Define the `TrainingArgument`

In [41]:
train_arg = TrainingArguments(
    output_dir="./emotion_classification",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,  # Experiment with slightly different learning rates
    per_device_train_batch_size=4,  # Increase batch size if possible
    per_device_eval_batch_size=4,
    num_train_epochs=20,  # Reduced number of epochs, we'll use early stopping
    warmup_ratio=0.1,  # Adjusted warmup ratio
    weight_decay=0.01,  # Adding weight decay for regularization
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    gradient_accumulation_steps=2,  # Simulate larger batch size
    fp16=True,  # Enable mixed precision training for faster performance
)

In [42]:
from transformers import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(early_stopping_patience=3)

# Call the `trainer API`

In [44]:
trainer=Trainer(
    model=model,
    args=train_arg,
    train_dataset=train_df,
    eval_dataset=test_df,
    compute_metrics=compute_matrics,
    tokenizer=feature_extractor,
    callbacks=[early_stopping]
)

In [45]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.352700,1.187278,0.583333
2,1.233800,1.376490,0.531250
3,0.630700,0.845540,0.704861
4,0.349600,0.879322,0.711806
5,0.076800,1.053678,0.746528
6,0.125900,1.157311,0.739583
7,0.066500,1.142313,0.791667
8,0.000300,1.304643,0.777778
9,0.000200,1.090984,0.795139
10,0.000100,1.004856,0.819444


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generatio

TrainOutput(global_step=2448, training_loss=0.25138989365446607, metrics={'train_runtime': 3269.0645, 'train_samples_per_second': 7.048, 'train_steps_per_second': 0.881, 'total_flos': 1.3275301989329142e+18, 'train_loss': 0.25138989365446607, 'epoch': 17.0})

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Emotion-Classification/model")
feature_extractor.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Emotion-Classification/tokenizer")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024}


['/content/drive/MyDrive/Colab Notebooks/Emotion-Classification/tokenizer/preprocessor_config.json']

In [46]:
# model.eval()

# Testing

In [50]:
!pip install soundfile

In [58]:
from transformers import AutoModelForAudioClassification,AutoFeatureExtractor,pipeline
import torch
import soundfile as sf

In [60]:
# # prompt: how we can load test the model in a new data new data is test_df[0]

model_path = "/content/drive/MyDrive/Colab Notebooks/Emotion-Classification/model"
feature_extractor_path = "/content/drive/MyDrive/Colab Notebooks/Emotion-Classification/tokenizer"

model = AutoModelForAudioClassification.from_pretrained(model_path)
feature_extractor = AutoFeatureExtractor.from_pretrained(feature_extractor_path)

# load the audio file
audio_input, sampling_rate =sf.read('/content/03-01-03-01-01-01-01.wav')
sampling_rate=16000
# preprocess the audio input
inputs = feature_extractor(audio_input, sampling_rate=sampling_rate, return_tensors="pt")


# # # Make prediction
with torch.no_grad():
    logits = model(**inputs).logits

# # # Get predicted class
predicted_class_idx = logits.argmax(-1).item()
predicted_class = model.config.id2label[predicted_class_idx]

# # # Get actual class
# actual_class = id2label[test_df[0]['label']]

print("Predicted class:", predicted_class)
# print("Actual class:", actual_class)


Predicted class: sad


In [52]:
import soundfile as sf

In [57]:
sampling_rate=16000
sampling_rate

16000

In [56]:
audio_input

array([0., 0., 0., ..., 0., 0., 0.])